* date:2024-08-09
* author: manyu Chen,aoyong Li
* E-mail: aoyong.li@outlook.com

In [1]:
import pandas as pd
import numpy as np 
import os
from pathlib import Path

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def extract_files_based_prefix(in_folder, prefix):  
    """  
    Extracts all CSV files starting with the specified prefix from the specified folder 
    and returns a list sorted by file name.    
    """  
    in_folder = Path(in_folder)  

    in_files = list(in_folder.glob("*.csv"))  
    list_files = []  
 
    for in_file in in_files:  
        if in_file.name.startswith(prefix):  
            list_files.append(in_file)  

    list_files.sort(key=lambda x: x.name)  
    return list_files  

In [4]:
folder_path = "D:/INSTALL_ANY/test data"
list_files = extract_files_based_prefix(folder_path,'output')
list_files

[WindowsPath('D:/INSTALL_ANY/test data/output2800.csv'),
 WindowsPath('D:/INSTALL_ANY/test data/output2801.csv'),
 WindowsPath('D:/INSTALL_ANY/test data/output2802.csv'),
 WindowsPath('D:/INSTALL_ANY/test data/output2803.csv'),
 WindowsPath('D:/INSTALL_ANY/test data/output2804.csv')]

In [5]:
def get_records(list_files):  
    """    
    """  
    list_dfs = []  
      
    for in_file in list_files:  
        if not os.path.exists(in_file):  
            print("The file does not exist:", in_file)  
            continue  
          
        # Read CSV file 
        df_tmp = pd.read_csv(in_file, delimiter=';')  
          
        # Remove duplicates based on 'Scooter ID' 
        df_tmp = df_tmp.drop_duplicates(subset=['Scooter ID'])  
          
        #  Extract numbers from file names and add 'ReqID' column  
        source_number = int(''.join(filter(str.isdigit, os.path.basename(in_file))))  
        df_tmp['ReqID'] = source_number  
          
        # Add processed DataFrame to list
        list_dfs.append(df_tmp)  
      
    # Merge all DataFrames   
    df = pd.concat(list_dfs, ignore_index=True)  
      
    # Calculate the number of missing values in each column and delete all NaN columns 
    nan_sum = df.isna().sum()  
    df = df.drop(columns=nan_sum[nan_sum == df.shape[0]].index)  
      
    # Sort the merged DataFrame (note: 'Scooter ID' and 'Timestamp' are used here)    
    df = df.sort_values(['Scooter ID', 'Timestamp'])  
      
    return df  

In [6]:
df = get_records(list_files)

In [7]:
df

,Provider,City,Scooter ID,lat,lng,Battery level,Price unlock,Price min,Timestamp,ReqID
10416,Bolt,Bochum,130137,51.698818,7.752383,86,0.00€,0.25€/MIN,2024-06-08 08:25:10.239713,2800
171059,Bolt,Bochum,130137,51.698803,7.752380,86,0.00€,0.25€/MIN,2024-06-08 08:30:13.618507,2801
331646,Bolt,Bochum,130137,51.698795,7.752370,86,0.00€,0.25€/MIN,2024-06-08 08:35:11.067868,2802
492184,Bolt,Bochum,130137,51.698803,7.752355,86,0.00€,0.25€/MIN,2024-06-08 08:40:11.686357,2803
652649,Bolt,Bochum,130137,51.698757,7.752328,86,0.00€,0.25€/MIN,2024-06-08 08:45:10.397145,2804
...,...,...,...,...,...,...,...,...,...,...
62567,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,1.2,0.25,2024-06-08 08:25:29.082360,2800
223158,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,1.2,0.25,2024-06-08 08:30:40.108973,2801
383713,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,1.2,0.25,2024-06-08 08:35:30.385697,2802
544241,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,1.2,0.25,2024-06-08 08:40:30.980175,2803


In [8]:
def process_scooter_data(df):  
    """  
    """  
    # Step 1: Remove 'Price' columns not used for our analysis  
    df = df.drop(['Price min', 'Price unlock'], axis=1)  
  
    # Step 2: Correct data types  
    # Ensure that 'Timestamp' is in datetime format for proper sorting  
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])  
  
    # Convert 'Battery level' to numeric, coercing errors to NaN  
    df['Battery level'] = pd.to_numeric(df['Battery level'], errors='coerce')  
  
    # Sort the DataFrame based on 'Scooter ID' and 'Timestamp' to ensure consistency  
    df = df.sort_values(by=['Scooter ID', 'Timestamp'])  
  
    # Drop duplicates based on 'Scooter ID' and 'ReqID', keeping the first occurrence  
    # Note: If you intended to use 'Timestamp' instead of 'ReqID', replace 'ReqID' with 'Timestamp'  
    df = df.drop_duplicates(subset=['Scooter ID', 'ReqID'], keep='first')  
  
    return df  

In [9]:
df = process_scooter_data(df)  

In [10]:
df

,Provider,City,Scooter ID,lat,lng,Battery level,Timestamp,ReqID
10416,Bolt,Bochum,130137,51.698818,7.752383,86,2024-06-08 08:25:10.239713,2800
171059,Bolt,Bochum,130137,51.698803,7.752380,86,2024-06-08 08:30:13.618507,2801
331646,Bolt,Bochum,130137,51.698795,7.752370,86,2024-06-08 08:35:11.067868,2802
492184,Bolt,Bochum,130137,51.698803,7.752355,86,2024-06-08 08:40:11.686357,2803
652649,Bolt,Bochum,130137,51.698757,7.752328,86,2024-06-08 08:45:10.397145,2804
...,...,...,...,...,...,...,...,...
62567,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,2024-06-08 08:25:29.082360,2800
223158,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,2024-06-08 08:30:40.108973,2801
383713,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,2024-06-08 08:35:30.385697,2802
544241,Tier,Aachen,ffff163f-e647-4f49-bc88-4eca2a82956c,50.780480,6.082869,73,2024-06-08 08:40:30.980175,2803


In [11]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the geographic distance (in kilometers) between two points
    """
    R = 6371  # 地球半径，单位：公里
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [12]:
def extract_trips(df):
    """
    Used to extract trips from the input DataFrame
    """
    df = df.sort_values(by=['Scooter ID', 'Timestamp'])
    
    df['lat'] = df['lat'].astype("float")
    df['lng'] = df['lng'].astype("float")
       
    for col in ['lat','lng','Timestamp','Battery level', 'ReqID','Scooter ID']:
        if col in df.columns:
            df[f'pre_{col}'] = df[col].shift(1)
            
    print(df.columns)
    df['dis_pre'] = haversine(df['pre_lng'], df['pre_lat'], df['lng'], df['lat']) * 1000
    df['dur_pre'] = (df['Timestamp'] - df['pre_Timestamp']).dt.total_seconds() / 60
    
    df_trips = df[(df['Scooter ID'] == df['pre_Scooter ID']) 
                & (df['dis_pre'] >= 100) &  (df['dis_pre'] <= 10000)
                  & (df['dur_pre'] <= 60)
                 & ((df['Provider'].isin(['Lime', 'Zeus'])) | (df['pre_Battery level'] >= df['Battery level']))].copy()

    df_trips = df_trips.rename(columns={
        "pre_ReqID": "ReqID Start", "ReqID":"ReqID End", "lng":"elng", "lat":"elat", 
        "pre_Timestamp" : "Start Time", "Timestamp": "End Time",
        "pre_lat": "Start Location Lat",
        "pre_lng": "Start Location Lng",
        "lat":'End Location Lat',
        "lng":'End Location Lng'})
    
    df_trips['Start Location'] = list(zip(df_trips['Start Location Lat'], df_trips['Start Location Lng']))
    df_trips['End Location'] = list(zip(df_trips['End Location Lat'], df_trips['End Location Lng']))
    
    df_trips = df_trips.drop(columns=['Start Location Lat', 'Start Location Lng', 
                        'End Location Lat', 'End Location Lng'])
    
    return df_trips

In [13]:
import time

# 记录开始时间
start_time = time.time()

# 运行识别出行记录的函数
df_trips = extract_trips(df)

# 记录结束时间
end_time = time.time()

# 计算并打印运行时间
print(f"Runtime: {end_time - start_time} seconds")

Index(['Provider', 'City', 'Scooter ID', 'lat', 'lng', 'Battery level',
       'Timestamp', 'ReqID', 'pre_lat', 'pre_lng', 'pre_Timestamp',
       'pre_Battery level', 'pre_ReqID', 'pre_Scooter ID'],
      dtype='object')
Runtime: 1.3305246829986572 seconds


In [14]:
df_trips

,Provider,City,Scooter ID,Battery level,End Time,ReqID End,Start Time,pre_Battery level,ReqID Start,pre_Scooter ID,dis_pre,dur_pre,Start Location,End Location
514462,Bolt,Karlsruhe,146112,89,2024-06-08 08:40:19.421942,2803,2024-06-08 08:30:22.657711,91.0,2801.0,146112,782.502188,9.946071,"(48.98583167, 8.363103333)","(48.97963667, 8.36819)"
675052,Bolt,Karlsruhe,150072,55,2024-06-08 08:45:19.114422,2804,2024-06-08 08:35:18.342678,56.0,2802.0,150072,434.935171,10.012862,"(49.00161833, 8.352955)","(48.99819833, 8.350061667)"
657203,Bolt,Darmstadt,150693,70,2024-06-08 08:45:12.466353,2804,2024-06-08 08:40:13.421321,73.0,2803.0,150693,805.009974,4.984084,"(50.10896683, 8.54179287)","(50.11446762, 8.534453392)"
496476,Bolt,Darmstadt,151236,28,2024-06-08 08:40:13.420204,2803,2024-06-08 08:30:15.406587,34.0,2801.0,151236,1909.325791,9.966894,"(50.12952805, 8.699305534)","(50.11906052, 8.678074837)"
169709,Bolt,Bochum,151281,68,2024-06-08 08:30:13.614734,2801,2024-06-08 08:25:10.235887,69.0,2800.0,151281,272.252885,5.056314,"(51.52811813, 7.402357101)","(51.53038025, 7.40085125)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785953,Voi,Koeln,ff3d9e84-a719-4d84-9592-fb966d3a488d,51,2024-06-08 08:46:15.329430,2804,2024-06-08 08:36:14.595572,55.0,2802.0,ff3d9e84-a719-4d84-9592-fb966d3a488d,1268.502887,10.012231,"(50.9330469, 7.0217572)","(50.9384124, 7.0057821)"
556534,Tier,Duisburg,ff563239-ffa8-45e2-9186-7c4f6ad68f01,43,2024-06-08 08:40:38.849016,2803,2024-06-08 08:35:37.628907,43.0,2802.0,ff563239-ffa8-45e2-9186-7c4f6ad68f01,187.295504,5.020335,"(51.419737, 6.754484)","(51.421409, 6.754157)"
727013,Tier,Hannover,ff6c6b79-e076-4c42-b023-956768f9eab7,71,2024-06-08 08:45:46.081472,2804,2024-06-08 08:40:46.163282,71.0,2803.0,ff6c6b79-e076-4c42-b023-956768f9eab7,204.981382,4.998636,"(52.372024, 9.705701)","(52.373844, 9.706181)"
475619,Voi,Augsburg,ff720d3f-ee6b-4993-ac51-1a436b517f0d,32,2024-06-08 08:36:16.430091,2802,2024-06-08 08:31:26.166792,34.0,2801.0,ff720d3f-ee6b-4993-ac51-1a436b517f0d,389.060229,4.837722,"(48.3655608, 10.8874206)","(48.3659846, 10.8821929)"


In [35]:
def format_and_merge_trips(df_trips, request_log_path):  
    # 首先加载request_log.csv文件  
    request_log_df = pd.read_csv(request_log_path, delimiter=';')  
      
    # 假设Start Location和End Location是形如(lat, lng)的元组  
    # 将Start Location和End Location拆分为单独的经纬度列  
    df_trips[['Start lat', 'Start lng']] = pd.DataFrame(df_trips['Start Location'].tolist(), index=df_trips.index)  
    df_trips[['End lat', 'End lng']] = pd.DataFrame(df_trips['End Location'].tolist(), index=df_trips.index)  
      
    # 选择并重命名列以匹配目标结构  
    formatted_trips_df = df_trips[['ReqID Start', 'ReqID End', 'Provider', 'Scooter ID', 'City', 'Start Time', 'End Time', 'Start lat', 'Start lng', 'End lat', 'End lng', 'dis_pre']]  
    formatted_trips_df.rename(columns={'Start Time': 'Start', 'End Time': 'End'}, inplace=True)  
      
    # 使用merge函数将request_log_df中的数据合并到formatted_trips_df中  
    formatted_trips_df = formatted_trips_df.merge(request_log_df[['ReqID', 'Temperature', 'WeatherID', 'City']],  
                                                  left_on=['ReqID Start', 'City'],  
                                                  right_on=['ReqID', 'City'],  
                                                  how='left')  
      
    # 删除合并后多余的ReqID列  
    formatted_trips_df.drop('ReqID', axis=1, inplace=True)  
      
    # 返回处理后的DataFrame  
    return formatted_trips_df  

In [34]:
request_log_path = 'D:/INSTALL_ANY/test data/request_log.csv'  
formatted_trips_df = format_and_merge_trips(df_trips, request_log_path) 

In [36]:
formatted_trips_df

,ReqID Start,ReqID End,Provider,Scooter ID,City,Start,End,Start lat,Start lng,End lat,End lng,dis_pre,Temperature,WeatherID
0,2801.0,2803,Bolt,146112,Karlsruhe,2024-06-08 08:30:22.657711,2024-06-08 08:40:19.421942,48.985832,8.363103,48.979637,8.368190,782.502188,15.00,0.0
1,2802.0,2804,Bolt,150072,Karlsruhe,2024-06-08 08:35:18.342678,2024-06-08 08:45:19.114422,49.001618,8.352955,48.998198,8.350062,434.935171,15.00,0.0
2,2803.0,2804,Bolt,150693,Darmstadt,2024-06-08 08:40:13.421321,2024-06-08 08:45:12.466353,50.108967,8.541793,50.114468,8.534453,805.009974,13.75,0.0
3,2801.0,2803,Bolt,151236,Darmstadt,2024-06-08 08:30:15.406587,2024-06-08 08:40:13.420204,50.129528,8.699306,50.119061,8.678075,1909.325791,13.75,0.0
4,2800.0,2801,Bolt,151281,Bochum,2024-06-08 08:25:10.235887,2024-06-08 08:30:13.614734,51.528118,7.402357,51.530380,7.400851,272.252885,13.40,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,2802.0,2804,Voi,ff3d9e84-a719-4d84-9592-fb966d3a488d,Koeln,2024-06-08 08:36:14.595572,2024-06-08 08:46:15.329430,50.933047,7.021757,50.938412,7.005782,1268.502887,12.40,0.0
1545,2802.0,2803,Tier,ff563239-ffa8-45e2-9186-7c4f6ad68f01,Duisburg,2024-06-08 08:35:37.628907,2024-06-08 08:40:38.849016,51.419737,6.754484,51.421409,6.754157,187.295504,12.85,1.0
1546,2803.0,2804,Tier,ff6c6b79-e076-4c42-b023-956768f9eab7,Hannover,2024-06-08 08:40:46.163282,2024-06-08 08:45:46.081472,52.372024,9.705701,52.373844,9.706181,204.981382,13.40,3.0
1547,2801.0,2802,Voi,ff720d3f-ee6b-4993-ac51-1a436b517f0d,Augsburg,2024-06-08 08:31:26.166792,2024-06-08 08:36:16.430091,48.365561,10.887421,48.365985,10.882193,389.060229,14.85,0.0
